In [1]:
#imports
import numpy as np
import utils
from dataset import Dataset
from feature_extractor import FeatureExtractor
from monitors_input import SHINE_monitor, SHINE_monitor2
from monitors_internals import MahalanobisMonitor, OutsideTheBoxMonitor, MaxSoftmaxProbabilityMonitor,\
                    MaxLogitMonitor, EnergyMonitor, ReActMonitor

2022-08-30 17:27:16.806386: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-30 17:27:16.806401: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#model params
batch_size = 10
model = "resnet"

#monitor params
layer_relu_ids = [32]
additional_transform = None
adversarial_attack = None#

#test dataset params
threat_type = 'distributional_shift'
variation = 'snow'
severity = 1
compl_path = '{}_severity_{}'.format(variation, severity)

#train dataset params
id_dataset = "cifar10"
num_classes_id = 10

#loading training set
#(X_train, y_train), (X_val, y_val) = load_dataset(
X_train, y_train = utils.load_dataset(
    threat_type, compl_path, id_dataset, 'train', root_path='../PRDC_2021_Data_profile_module/data')#utils.load_data(id_dataset, None)
X_train = X_train.astype('float32')
#X_val = X_val.astype('float32')
print(np.shape(X_train), np.shape(y_train))
#print(np.shape(X_val), np.shape(y_val))

loading data from ../PRDC_2021_Data_profile_module/data/training_set/distributional_shift/cifar10/snow_severity_1
(50000, 32, 32, 3) (50000,)


In [3]:
dataset_train = Dataset(compl_path, "train", model, (X_train, y_train), batch_size=batch_size)

In [4]:
#extracting and storing features from test data using an ML model trained on ID data
feature_extractor = FeatureExtractor(model, id_dataset, layer_relu_ids)
features_train, logits_train, softmax_train, pred_train, lab_train = feature_extractor.get_features(dataset_train)

In [5]:
# for the monitors that use data from activation function layers
id_layer_monitored = -1

In [ ]:
# building SHINE monitor
# it can take a lot of time
monitor_shine = SHINE_monitor(id_dataset)
monitor_shine.fit(X_train, y_train, features_train[id_layer_monitored], pred_train)
monitor_shine.name = 'shine'
utils.save_monitors([monitor_shine], id_dataset)
print('number of image density estimators',len(monitor_shine.arr_density_img))
print('number of group of representative feature instances',len(monitor_shine.arr_repr_feature_TP))

In [ ]:
# building oob monitor
monitor_oob = OutsideTheBoxMonitor(n_clusters=3)
monitor_oob.name = 'oob'
monitor_oob.fit(features_train[id_layer_monitored], lab_train)

In [ ]:
# building react monitor
monitor_react = ReActMonitor(quantile_value=0.99, mode='msp')
monitor_react.name = 'react'
monitor_react.fit(feature_extractor, features_train[id_layer_monitored])

In [ ]:
# building softmax monitor
monitor_msp = MaxSoftmaxProbabilityMonitor()
monitor_msp.name = 'msp'
monitor_msp.fit()

In [ ]:
# building max logit monitor
monitor_maxlogits = MaxLogitMonitor()
monitor_maxlogits.name = 'maxlogit'
monitor_maxlogits.fit()

In [ ]:
# building Energy monitor
T = 1
monitor_energy = EnergyMonitor(temperature=T)
monitor_energy.name = 'energy'
monitor_energy.fit()

In [ ]:
# building mahalanobis monitor
monitor_mahalanobis = MahalanobisMonitor(id_dataset, model, id_layer_monitored, is_tied=True)
monitor_mahalanobis.name = 'mahalanobis'
monitor_mahalanobis.fit(features_train[id_layer_monitored], lab_train)

In [ ]:
# defining thresholds for the monitors based on score
# such thresholds are respecting a limit of 95% of scores on correct classified instances on ID dataset
arr_thresholds_react = []
arr_thresholds_msp = []
arr_thresholds_logits = []
arr_thresholds_energy = []
arr_thresholds_mahalanobis = []
arr_thresholds_shine = {'S':[], 'HINE':[]}

for label in range(num_classes_id):
    ind_cls = np.where(y_train==label) # for each class
    ind_correct = np.where(y_train[ind_cls]==pred_train[ind_cls]) #take the correct classified instances
    ind_incorrect = np.where(y_train[ind_cls]!=pred_train[ind_cls]) #take the INcorrect classified instances
    
    threshold = utils.calculate_threshold(monitor_react, label, features_train[id_layer_monitored][ind_correct])
    arr_thresholds_react.append(threshold)
    
    threshold = utils.calculate_threshold(monitor_msp, label, softmax_train[ind_correct])
    arr_thresholds_msp.append(threshold)
    
    threshold = utils.calculate_threshold(monitor_maxlogits, label, logits_train[ind_correct])
    arr_thresholds_logits.append(threshold)
    
    threshold = utils.calculate_threshold(monitor_energy, label, logits_train[ind_correct])
    arr_thresholds_energy.append(threshold)
    
    threshold = utils.calculate_threshold(monitor_mahalanobis, label, features_train[id_layer_monitored][ind_correct])
    arr_thresholds_mahalanobis.append(threshold)
    
    # SHINE uses info from both correct (features) and incorrect (images) classified instances from a class
    threshold_S, threshold_HINE = utils.calculate_threshold_SHINE(monitor_shine, X_train[ind_incorrect], label, 
                                                                  pred_train[ind_cls], 
                                                                  features_train[id_layer_monitored][ind_cls])
    arr_thresholds_shine['S'].append(threshold_S)
    arr_thresholds_shine['HINE'].append(threshold_HINE)
    
    
# attributing array of calculated thresholds for each monitor
monitor_react.arr_thresholds = arr_thresholds_react
monitor_msp.arr_thresholds = arr_thresholds_msp
monitor_maxlogits.arr_thresholds = arr_thresholds_logits
monitor_energy.arr_thresholds = arr_thresholds_energy
monitor_mahalanobis.arr_thresholds = arr_thresholds_mahalanobis
monitor_shine.arr_thresholds = arr_thresholds_shine

In [ ]:
#saving monitors
arr_monitors = []
arr_monitors.append(monitor_oob)
arr_monitors.append(monitor_react)
arr_monitors.append(monitor_msp)
arr_monitors.append(monitor_maxlogits)
arr_monitors.append(monitor_energy)
arr_monitors.append(monitor_mahalanobis)
arr_monitors.append(monitor_shine)

utils.save_monitors(arr_monitors, id_dataset)

In [15]:
# OPTIONAL: test if the monitors were correctly saved
import os
import pickle
monitor_name = 'react'
num_instances = 10 # testing with 10 instances
monitor_filename = os.path.join('Monitors', id_dataset, '{}.sav'.format(monitor_name))
monitor = pickle.load(open(monitor_filename, 'rb'))

for x, pred, feature, softmax, logits, label in zip(X_train[:num_instances], pred_train[:num_instances], 
                                                    features_train[id_layer_monitored][:num_instances],
                                                 softmax_train[:num_instances], logits_train[:num_instances],
                                                    y_train[:num_instances]):
    
    m_score = monitor.predict(np.array([feature]))[0]
    m_pred = m_score<=monitor.arr_thresholds[pred]
    print('monitor reaction', m_pred)

monitor reaction True
monitor reaction True
monitor reaction True
monitor reaction False
monitor reaction True
monitor reaction True
monitor reaction True
monitor reaction True
monitor reaction True
monitor reaction True


In [14]:
# OPTIONAL: test if SHINE monitor was correctly saved
import os
import pickle
monitor_name = 'shine'
num_instances = 10 # testing with 10 instances
monitor_filename = os.path.join('Monitors', id_dataset, '{}.sav'.format(monitor_name))
monitor = pickle.load(open(monitor_filename, 'rb'))

for x, pred, feature, softmax, logits, label in zip(X_train[:num_instances], pred_train[:num_instances], 
                                                    features_train[id_layer_monitored][:num_instances],
                                                 softmax_train[:num_instances], logits_train[:num_instances],
                                                    y_train[:num_instances]):
    
    
    avg_sim_TP, logprob = monitor.predict(np.array([x]), feature, pred)
    if avg_sim_TP >= monitor.arr_thresholds['S'][pred]['TP']:
        print(False) 
    elif avg_sim_TP <= monitor.arr_thresholds['S'][pred]['FN']:
        print(True)
    else:
        print(np.exp(logprob) <= monitor.arr_thresholds['HINE'][pred])

(1, 32, 32, 3)
False
(1, 32, 32, 3)
True
(1, 32, 32, 3)
True
(1, 32, 32, 3)
True
(1, 32, 32, 3)
True
(1, 32, 32, 3)
True
(1, 32, 32, 3)
True
(1, 32, 32, 3)
True
(1, 32, 32, 3)
True
(1, 32, 32, 3)
False
